In [1]:
%load_ext Cython
x_min = -2
x_max = 1
y_min = -1.5
y_max = 1.5
width = 4000
height = 4000
max_iter = 100000

In [2]:
import numpy as np
import time
from random import uniform
import threading
import concurrent
import multiprocessing
from math import pi, sin, cos
import hashlib


class buddhabrot:

    def __init__(self, x_min, x_max, y_min, y_max, width, height, max_iter):
        self.x_min = x_min # real min bound
        self.x_max = x_max # real max bound
        self.y_min = y_min # imaginary min bound
        self.y_max = y_max # imaginary max bound
        self.width = width # pixel width
        self.height = height # pixel height
        self.max_iter = max_iter # maximum iterations per point
        self.file_name = f"buddhabrot_{x_min}_{x_max}_{y_min}_{y_max}_{width}_{height}_{max_iter}"
        self.np_file_name = self.file_name + '.npy'
        self.log_file_name = self.file_name + '.txt'
        try:
            self.grid_array = np.load(self.np_file_name)
            self.print_update('Old file found and loaded...')
        except FileNotFoundError:
            self.grid_array = np.zeros((width,height), np.int64)
            np.save(self.np_file_name, self.grid_array)
            self.print_update('No numpy file found. New file created...')
        try:
            with open(self.log_file_name, 'r') as f:
                self.num_eval_points = eval(f.read())
                self.print_update('Log file found and loaded...')
                self.print_update(str(self.num_eval_points) + ' points evaluated so far...')
        except FileNotFoundError:
            with open(self.log_file_name, 'w') as f:
                self.print_update('No log file found. New file created...')
                self.print_update('Estimate evaluated points...')
                self.num_eval_points = self.grid_array.sum() // self.max_iter
                self.print_update(str(self.num_eval_points) + ' points estimated...')
                f.write(str(self.num_eval_points))
    
    def print_update(self, msg):
        print(time.ctime() + ': ' + msg)
    
    def get_grid_point(self, z_real, z_imag):
        ###
        # compute pixel density
        x_dens = (self.width - 1) / abs(self.x_max - self.x_min)
        y_dens = (self.height - 1) / abs(self.y_max - self.y_min)
        ###
        # compute nearest pixel
        x_pixel = round(x_dens * (z_real - self.x_min))
        y_pixel = self.width - 1 - round(y_dens * (z_imag - self.y_min)) # image starts top left
        ###
        # does nearest pixel exceed image size?
        if x_pixel < 0 or x_pixel > self.width -1 or y_pixel < 0 or y_pixel > self.height - 1:
            return None
        
        return (x_pixel, y_pixel)
    
    def create_rand_point(self):
        ###
        # polar coordinates
        r = uniform(0, 2)
        phi = uniform(0, 2*pi)
        return (r*cos(phi), r*sin(phi))
    
    def worker(self, loops):
        ###
        # establish array
        grid_array = np.zeros((self.width, self.height), dtype = np.int64)
        ###
        # iterate over points for this worker
        for _ in range(loops):
            # add point to grid
            self.add_point_to_grid(grid_array)
            
        return grid_array
    
    def add_point_to_grid(self, grid_array):
        ###
        # create random point
        (c_real, c_imag) = self.create_rand_point()
        ###
        # first step
        z_real = c_real
        z_imag = c_imag
        ###
        # establish path
        path = {}
        ###
        # start iteration
        for n in range(self.max_iter):
            z_real2 = z_real * z_real
            z_imag2 = z_imag * z_imag
            ###
            # does zn surely diverge?
            if z_real2 + z_imag2 > 4.0:
                for (x,y) in path:
                    grid_array[x,y] += path[(x,y)]
                break
            ###
            # get nearest pixel
            grid_point = self.get_grid_point(z_real, z_imag)
            ###
            # pixel already visited?
            if grid_point in path:
                    path[grid_point] += 1
            elif grid_point is not None:
                path[grid_point] = 1
            ###
            # prepare new step
            z_imag = 2 * z_real*z_imag + c_imag
            z_real = z_real2 - z_imag2 + c_real

    def get_array(self, update=True):
        ###
        # update from file
        if update:
            self.grid_array = np.load(self.np_file_name)
        return self.grid_array
    
    def load_np_file(self):
        self.grid_array = np.load(self.np_file_name)
        self.print_update('From file updated...')
        
    def save_np_file(self):
        np.save(self.file_name, self.grid_array)
        self.print_update('File saved...')
            
    def sharpen_buddha(self, loops=1, num_workers=1):
        ###
        # start multiprocessing
        with concurrent.futures.ProcessPoolExecutor() as executor:
            ###
            # create workers
            workers = [executor.submit(buddhabrot.worker, self, loops) for _ in range(num_workers)]
            ###
            # combine results
            for f in concurrent.futures.as_completed(workers):
                self.grid_array += f.result()
                self.num_eval_points += loops
                self.save_np_file()
                with open(self.log_file_name, 'w') as f:
                    f.write(str(self.num_eval_points))
                self.print_update(f"{loops} new points evaluated and saved...")

In [ ]:
%%time
buddha = buddhabrot(x_min, x_max, y_min, y_max, width, height, max_iter)
buddha.sharpen_buddha(loops=1000, num_workers=1000)
array = buddha.get_array()

Mon Mar 30 14:42:07 2020: Old file found and loaded...
Mon Mar 30 14:42:07 2020: Log file found and loaded...
Mon Mar 30 14:42:07 2020: 90000 points evaluated so far...
Mon Mar 30 14:45:14 2020: File saved...
Mon Mar 30 14:45:14 2020: 1000 new points evaluated and saved...
Mon Mar 30 14:45:15 2020: File saved...
Mon Mar 30 14:45:15 2020: 1000 new points evaluated and saved...
Mon Mar 30 14:45:16 2020: File saved...
Mon Mar 30 14:45:16 2020: 1000 new points evaluated and saved...
Mon Mar 30 14:45:17 2020: File saved...
Mon Mar 30 14:45:17 2020: 1000 new points evaluated and saved...
Mon Mar 30 14:45:18 2020: File saved...
Mon Mar 30 14:45:18 2020: 1000 new points evaluated and saved...
Mon Mar 30 14:45:22 2020: File saved...
Mon Mar 30 14:45:22 2020: 1000 new points evaluated and saved...
Mon Mar 30 14:45:22 2020: File saved...
Mon Mar 30 14:45:22 2020: 1000 new points evaluated and saved...
Mon Mar 30 14:45:28 2020: File saved...
Mon Mar 30 14:45:28 2020: 1000 new points evaluated and 

In [ ]:
%%cython
from PIL import Image

cpdef create_image(array):
    (width, height) = array.shape
    img = Image.new('RGB', (width, height))
    max_val = array.max()
    pixels = []
    for y in range(height):
        for x in range(width):
            gray = int(255 * (1 - array[x,y]/max_val))
            pixels.append((gray, gray, gray))
    img.putdata(pixels)
    return img

In [ ]:
img = create_image(array)

In [ ]:
img.show()

In [ ]:
img.save('buddhabrot_' + str(width) + '_' + str(height) + '_' + str(max_iter) + '.png')

In [ ]:
buddha.get_array()

In [ ]:
%%time
import hashlib

block_size = 65536
file_hash = hashlib.sha256()
with open('buddhabrot_-2_1_-1.5_1.5_4000_4000_100000.npy', 'rb') as f:
    f_block = f.read(block_size)
    while len(f_block) > 0:
        file_hash.update(f_block)
        f_block = f.read(block_size)
print(file_hash.hexdigest())